In [4]:
from bs4 import BeautifulSoup
import io
import tiktoken

In [6]:
def extract_text(filename):
    f = io.open(filename, mode="r", encoding="utf-8")
    html_doc = f.read()
    f.close()
    soup = BeautifulSoup(html_doc, 'html.parser')
    text = soup.get_text()
    return text

In [ ]:
def chunking(text, max_token_count):
    enc = tiktoken.get_encoding("cl100k_base")

    words = text.split()
    chunk = []
    encodeStr = ""
    token_count = 0
    
    
    for word in words:
        if token_count+len(enc.encode(word))>max_token_count:
            chunk.append(encodeStr.strip())
            encodeStr = ""
            token_count = 0
        else:
            encodeStr = encodeStr + " " + word
            token_count += len(enc.encode(word))
    
    if encodeStr:        
        chunk.append(encodeStr.strip())
        
    return chunk

In [ ]:
def extract_and_chunk_text(directory, max_token_count):
    results = {}
    
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.html'):
                file_path = os.path.join(dirpath, filename)
                text = extract_text(file_path)
                chunks = chunking(text, max_tokens)
                results[file_path] = chunks
                
                
    return results